In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.9))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [7]:
suffix = 'sim09.csv'
n_samples = 500

### BERT-based-uncased

In [8]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7317073170731707
KNN 0.6801125703564728
SVC 0.8030018761726079


In [10]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [11]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [12]:
runAttack(bert_model_wrapper_lr_mr, n_samples, 'attack_results_new/bertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:   1%|▎                           | 5/500 [00:24<41:09,  4.99s/it]textattack: Saving checkpoint under "checkpoints\1720258035893.ta.chkpt" at 2024-07-06 17:27:15 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 5 / 4 / 10:   2%|▌                         | 10/500 [00:29<23:47,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1720258040082.ta.chkpt" at 2024-07-06 17:27:20 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 10 / 4 / 15:   3%|▊                        | 15/500 [00:36<19:43,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1720258047564.ta.chkpt" at 2024-07-06 17:27:27 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 110 / 276 / 99 / 485:  97%|██████████████████▍| 485/500 [09:14<00:17,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720258565503.ta.chkpt" at 2024-07-06 17:36:05 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 111 / 279 / 100 / 490:  98%|█████████████████▋| 490/500 [09:19<00:11,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720258570476.ta.chkpt" at 2024-07-06 17:36:10 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 114 / 281 / 100 / 495:  99%|█████████████████▊| 495/500 [09:27<00:05,  1.15s/it]textattack: Saving checkpoint under "checkpoints\1720258578739.ta.chkpt" at 2024-07-06 17:36:18 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 115 / 285 / 100 / 500: 100%|██████████████████| 500/500 [09:34<00:00,  1.15s/it]textattack: Saving checkpoint under "checkpoints\1720258585673.ta.chkpt" at 2024-07-06 17:36:25 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 115 / 285 / 100 / 500: 100%|██████████████████| 500/500 [09:34<00:00,  1.15s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 115    |
| Number of failed attacks:     | 285    |
| Number of skipped attacks:    | 100    |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 57.0%  |
| Attack success rate:          | 28.75% |
| Average perturbed word %:     | 13.81% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 22.58  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_rf_mr, n_samples, 'attack_results_new/bertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                           | 5/500 [00:11<18:45,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720258597158.ta.chkpt" at 2024-07-06 17:36:37 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:   1%|▍                           | 7/500 [00:11<13:33,  1.65s/it]

[Succeeded / Failed / Skipped / Total] 1 / 5 / 4 / 10:   2%|▌                         | 10/500 [00:16<13:17,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1720258602063.ta.chkpt" at 2024-07-06 17:36:42 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   3%|▊                         | 15/500 [00:24<13:13,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1720258610343.ta.chkpt" at 2024-07-06 17:36:50 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 145 / 204 / 136 / 485:  97%|█████████████████▍| 485/500 [08:01<00:14,  1.01it/s]textattack: Saving checkpoint under "checkpoints\1720259067508.ta.chkpt" at 2024-07-06 17:44:27 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 147 / 206 / 137 / 490:  98%|█████████████████▋| 490/500 [08:07<00:09,  1.01it/s]textattack: Saving checkpoint under "checkpoints\1720259073117.ta.chkpt" at 2024-07-06 17:44:33 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 147 / 206 / 138 / 491:  98%|█████████████████▋| 491/500 [08:07<00:08,  1.01it/s]

[Succeeded / Failed / Skipped / Total] 148 / 209 / 138 / 495:  99%|█████████████████▊| 495/500 [08:16<00:05,  1.00s/it]textattack: Saving checkpoint under "checkpoints\1720259081789.ta.chkpt" at 2024-07-06 17:44:41 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 151 / 210 / 139 / 500: 100%|██████████████████| 500/500 [08:19<00:00,  1.00it/s]textattack: Saving checkpoint under "checkpoints\1720259085275.ta.chkpt" at 2024-07-06 17:44:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 151 / 210 / 139 / 500: 100%|██████████████████| 500/500 [08:19<00:00,  1.00it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 151    |
| Number of failed attacks:     | 210    |
| Number of skipped attacks:    | 139    |
| Original accuracy:            | 72.2%  |
| Accuracy under attack:        | 42.0%  |
| Attack success rate:          | 41.83% |
| Average perturbed word %:     | 14.02% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 21.82  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_knn_mr, n_samples, 'attack_results_new/bertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:09<15:13,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1720259094580.ta.chkpt" at 2024-07-06 17:44:54 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▎                           | 6/500 [00:09<12:49,  1.56s/it]

[Succeeded / Failed / Skipped / Total] 5 / 1 / 4 / 10:   2%|▌                         | 10/500 [00:13<10:46,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720259098546.ta.chkpt" at 2024-07-06 17:44:58 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 4 / 6 / 15:   3%|▊                         | 15/500 [00:20<11:15,  1.39s/it]textattack: Saving checkpoint under "checkpoints\1720259106233.ta.chkpt" at 2024-07-06 17:45:06 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 243 / 95 / 147 / 485:  97%|██████████████████▍| 485/500 [06:34<00:12,  1.23it/s]textattack: Saving checkpoint under "checkpoints\1720259479657.ta.chkpt" at 2024-07-06 17:51:19 after 485 attacks.
[Succeeded / Failed / Skipped / Total] 243 / 95 / 148 / 486:  97%|██████████████████▍| 486/500 [06:34<00:11,  1.23it/s]

[Succeeded / Failed / Skipped / Total] 244 / 97 / 149 / 490:  98%|██████████████████▌| 490/500 [06:38<00:08,  1.23it/s]textattack: Saving checkpoint under "checkpoints\1720259484235.ta.chkpt" at 2024-07-06 17:51:24 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 244 / 97 / 150 / 491:  98%|██████████████████▋| 491/500 [06:39<00:07,  1.23it/s]

[Succeeded / Failed / Skipped / Total] 246 / 98 / 151 / 495:  99%|██████████████████▊| 495/500 [06:42<00:04,  1.23it/s]textattack: Saving checkpoint under "checkpoints\1720259487782.ta.chkpt" at 2024-07-06 17:51:27 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 248 / 99 / 153 / 500: 100%|███████████████████| 500/500 [06:45<00:00,  1.23it/s]textattack: Saving checkpoint under "checkpoints\1720259490782.ta.chkpt" at 2024-07-06 17:51:30 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 248 / 99 / 153 / 500: 100%|███████████████████| 500/500 [06:45<00:00,  1.23it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 248    |
| Number of failed attacks:     | 99     |
| Number of skipped attacks:    | 153    |
| Original accuracy:            | 69.4%  |
| Accuracy under attack:        | 19.8%  |
| Attack success rate:          | 71.47% |
| Average perturbed word %:     | 9.22%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 18.93  |
+-------------------------------+--------+


In [15]:
runAttack(bert_model_wrapper_svc_mr, n_samples, 'attack_results_new/bertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvcsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:   1%|▎                           | 5/500 [00:10<17:55,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1720259501723.ta.chkpt" at 2024-07-06 17:51:41 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 4 / 3 / 7:   1%|▍                           | 7/500 [00:11<12:57,  1.58s/it]

[Succeeded / Failed / Skipped / Total] 1 / 5 / 4 / 10:   2%|▌                         | 10/500 [00:15<12:46,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1720259506504.ta.chkpt" at 2024-07-06 17:51:46 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 10 / 4 / 15:   3%|▊                        | 15/500 [00:23<12:49,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720259514661.ta.chkpt" at 2024-07-06 17:51:54 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 118 / 271 / 96 / 485:  97%|██████████████████▍| 485/500 [09:08<00:16,  1.13s/it]textattack: Saving checkpoint under "checkpoints\1720260039478.ta.chkpt" at 2024-07-06 18:00:39 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 119 / 274 / 97 / 490:  98%|██████████████████▌| 490/500 [09:16<00:11,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720260047447.ta.chkpt" at 2024-07-06 18:00:47 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 121 / 277 / 97 / 495:  99%|██████████████████▊| 495/500 [09:24<00:05,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720260055792.ta.chkpt" at 2024-07-06 18:00:55 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 122 / 281 / 97 / 500: 100%|███████████████████| 500/500 [09:32<00:00,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720260062886.ta.chkpt" at 2024-07-06 18:01:02 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 122 / 281 / 97 / 500: 100%|███████████████████| 500/500 [09:32<00:00,  1.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 122    |
| Number of failed attacks:     | 281    |
| Number of skipped attacks:    | 97     |
| Original accuracy:            | 80.6%  |
| Accuracy under attack:        | 56.2%  |
| Attack success rate:          | 30.27% |
| Average perturbed word %:     | 15.01% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 22.69  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [16]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [18]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [19]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [20]:
runAttack(disbert_model_wrapper_lr_mr, n_samples, 'attack_results_new/disbertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                           | 5/500 [00:08<14:27,  1.75s/it]textattack: Saving checkpoint under "checkpoints\1720260219564.ta.chkpt" at 2024-07-06 18:03:39 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:   1%|▍                           | 7/500 [00:08<10:25,  1.27s/it]

[Succeeded / Failed / Skipped / Total] 2 / 5 / 3 / 10:   2%|▌                         | 10/500 [00:12<10:16,  1.26s/it]textattack: Saving checkpoint under "checkpoints\1720260223376.ta.chkpt" at 2024-07-06 18:03:43 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   3%|▊                         | 15/500 [00:16<09:08,  1.13s/it]textattack: Saving checkpoint under "checkpoints\1720260227767.ta.chkpt" at 2024-07-06 18:03:47 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 133 / 250 / 102 / 485:  97%|█████████████████▍| 485/500 [05:45<00:10,  1.41it/s]textattack: Saving checkpoint under "checkpoints\1720260555882.ta.chkpt" at 2024-07-06 18:09:15 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 136 / 251 / 103 / 490:  98%|█████████████████▋| 490/500 [05:47<00:07,  1.41it/s]textattack: Saving checkpoint under "checkpoints\1720260558320.ta.chkpt" at 2024-07-06 18:09:18 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 136 / 251 / 104 / 491:  98%|█████████████████▋| 491/500 [05:47<00:06,  1.41it/s]

[Succeeded / Failed / Skipped / Total] 137 / 254 / 104 / 495:  99%|█████████████████▊| 495/500 [05:52<00:03,  1.40it/s]textattack: Saving checkpoint under "checkpoints\1720260563773.ta.chkpt" at 2024-07-06 18:09:23 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 138 / 258 / 104 / 500: 100%|██████████████████| 500/500 [05:57<00:00,  1.40it/s]textattack: Saving checkpoint under "checkpoints\1720260568727.ta.chkpt" at 2024-07-06 18:09:28 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 138 / 258 / 104 / 500: 100%|██████████████████| 500/500 [05:57<00:00,  1.40it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 138    |
| Number of failed attacks:     | 258    |
| Number of skipped attacks:    | 104    |
| Original accuracy:            | 79.2%  |
| Accuracy under attack:        | 51.6%  |
| Attack success rate:          | 34.85% |
| Average perturbed word %:     | 13.81% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 22.21  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_rf_mr, n_samples, 'attack_results_new/disbertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:   1%|▎                           | 5/500 [00:09<15:36,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1720260578281.ta.chkpt" at 2024-07-06 18:09:38 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 4 / 2 / 6:   1%|▎                           | 6/500 [00:09<13:04,  1.59s/it]

[Succeeded / Failed / Skipped / Total] 1 / 7 / 2 / 10:   2%|▌                         | 10/500 [00:14<11:29,  1.41s/it]textattack: Saving checkpoint under "checkpoints\1720260582891.ta.chkpt" at 2024-07-06 18:09:42 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 10 / 3 / 15:   3%|▊                        | 15/500 [00:18<09:56,  1.23s/it]textattack: Saving checkpoint under "checkpoints\1720260587266.ta.chkpt" at 2024-07-06 18:09:47 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 135 / 233 / 117 / 485:  97%|█████████████████▍| 485/500 [05:49<00:10,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720260918302.ta.chkpt" at 2024-07-06 18:15:18 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 136 / 236 / 118 / 490:  98%|█████████████████▋| 490/500 [05:53<00:07,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720260922131.ta.chkpt" at 2024-07-06 18:15:22 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 136 / 236 / 119 / 491:  98%|█████████████████▋| 491/500 [05:53<00:06,  1.39it/s]

[Succeeded / Failed / Skipped / Total] 138 / 237 / 120 / 495:  99%|█████████████████▊| 495/500 [05:55<00:03,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720260924600.ta.chkpt" at 2024-07-06 18:15:24 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 140 / 240 / 120 / 500: 100%|██████████████████| 500/500 [06:01<00:00,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720260929835.ta.chkpt" at 2024-07-06 18:15:29 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 140 / 240 / 120 / 500: 100%|██████████████████| 500/500 [06:01<00:00,  1.38it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 140    |
| Number of failed attacks:     | 240    |
| Number of skipped attacks:    | 120    |
| Original accuracy:            | 76.0%  |
| Accuracy under attack:        | 48.0%  |
| Attack success rate:          | 36.84% |
| Average perturbed word %:     | 13.31% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 22.13  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_knn_mr, n_samples, 'attack_results_new/disbertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:10<16:49,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1720260940155.ta.chkpt" at 2024-07-06 18:15:40 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 3 / 2 / 7:   1%|▍                           | 7/500 [00:10<12:07,  1.48s/it]

[Succeeded / Failed / Skipped / Total] 3 / 5 / 2 / 10:   2%|▌                         | 10/500 [00:13<11:11,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720260943670.ta.chkpt" at 2024-07-06 18:15:43 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 8 / 3 / 15:   3%|▊                         | 15/500 [00:18<09:48,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720260948155.ta.chkpt" at 2024-07-06 18:15:48 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 165 / 192 / 128 / 485:  97%|█████████████████▍| 485/500 [05:23<00:09,  1.50it/s]textattack: Saving checkpoint under "checkpoints\1720261253208.ta.chkpt" at 2024-07-06 18:20:53 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 168 / 193 / 129 / 490:  98%|█████████████████▋| 490/500 [05:25<00:06,  1.50it/s]textattack: Saving checkpoint under "checkpoints\1720261255833.ta.chkpt" at 2024-07-06 18:20:55 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 169 / 195 / 131 / 495:  99%|█████████████████▊| 495/500 [05:29<00:03,  1.50it/s]textattack: Saving checkpoint under "checkpoints\1720261259192.ta.chkpt" at 2024-07-06 18:20:59 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 169 / 195 / 132 / 496:  99%|█████████████████▊| 496/500 [05:29<00:02,  1.51it/s]

[Succeeded / Failed / Skipped / Total] 169 / 199 / 132 / 500: 100%|██████████████████| 500/500 [05:33<00:00,  1.50it/s]textattack: Saving checkpoint under "checkpoints\1720261263755.ta.chkpt" at 2024-07-06 18:21:03 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 169 / 199 / 132 / 500: 100%|██████████████████| 500/500 [05:33<00:00,  1.50it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 169    |
| Number of failed attacks:     | 199    |
| Number of skipped attacks:    | 132    |
| Original accuracy:            | 73.6%  |
| Accuracy under attack:        | 39.8%  |
| Attack success rate:          | 45.92% |
| Average perturbed word %:     | 9.55%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 20.53  |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_svc_mr, n_samples, 'attack_results_new/disbertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvcsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                           | 5/500 [00:09<15:13,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1720261273072.ta.chkpt" at 2024-07-06 18:21:13 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:   1%|▍                           | 7/500 [00:09<10:58,  1.34s/it]

[Succeeded / Failed / Skipped / Total] 2 / 5 / 3 / 10:   2%|▌                         | 10/500 [00:13<10:41,  1.31s/it]textattack: Saving checkpoint under "checkpoints\1720261276932.ta.chkpt" at 2024-07-06 18:21:16 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   3%|▊                         | 15/500 [00:17<09:25,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720261281354.ta.chkpt" at 2024-07-06 18:21:21 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 140 / 241 / 104 / 485:  97%|█████████████████▍| 485/500 [05:49<00:10,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720261612929.ta.chkpt" at 2024-07-06 18:26:52 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 142 / 242 / 106 / 490:  98%|█████████████████▋| 490/500 [05:51<00:07,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720261615441.ta.chkpt" at 2024-07-06 18:26:55 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 142 / 242 / 107 / 491:  98%|█████████████████▋| 491/500 [05:51<00:06,  1.40it/s]

[Succeeded / Failed / Skipped / Total] 143 / 245 / 107 / 495:  99%|█████████████████▊| 495/500 [05:57<00:03,  1.39it/s]textattack: Saving checkpoint under "checkpoints\1720261621223.ta.chkpt" at 2024-07-06 18:27:01 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 144 / 249 / 107 / 500: 100%|██████████████████| 500/500 [06:02<00:00,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720261626332.ta.chkpt" at 2024-07-06 18:27:06 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 144 / 249 / 107 / 500: 100%|██████████████████| 500/500 [06:02<00:00,  1.38it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 144    |
| Number of failed attacks:     | 249    |
| Number of skipped attacks:    | 107    |
| Original accuracy:            | 78.6%  |
| Accuracy under attack:        | 49.8%  |
| Attack success rate:          | 36.64% |
| Average perturbed word %:     | 14.13% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 21.92  |
+-------------------------------+--------+


### CLIP

In [24]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [26]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [27]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [28]:
runAttack(clip_model_wrapper_lr_mr, n_samples, 'attack_results_new/clipmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   1%|▎                           | 5/500 [00:09<16:17,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1720262031998.ta.chkpt" at 2024-07-06 18:33:51 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 4 / 1 / 6:   1%|▎                           | 6/500 [00:09<13:43,  1.67s/it]

[Succeeded / Failed / Skipped / Total] 5 / 4 / 1 / 10:   2%|▌                         | 10/500 [00:13<11:10,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720262035811.ta.chkpt" at 2024-07-06 18:33:55 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 7 / 1 / 15:   3%|▊                         | 15/500 [00:19<10:23,  1.29s/it]textattack: Saving checkpoint under "checkpoints\1720262041421.ta.chkpt" at 2024-07-06 18:34:01 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 147 / 211 / 127 / 485:  97%|█████████████████▍| 485/500 [05:30<00:10,  1.47it/s]textattack: Saving checkpoint under "checkpoints\1720262352306.ta.chkpt" at 2024-07-06 18:39:12 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 148 / 213 / 129 / 490:  98%|█████████████████▋| 490/500 [05:32<00:06,  1.47it/s]textattack: Saving checkpoint under "checkpoints\1720262354556.ta.chkpt" at 2024-07-06 18:39:14 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 148 / 213 / 130 / 491:  98%|█████████████████▋| 491/500 [05:32<00:06,  1.48it/s]

[Succeeded / Failed / Skipped / Total] 148 / 216 / 131 / 495:  99%|█████████████████▊| 495/500 [05:35<00:03,  1.47it/s]textattack: Saving checkpoint under "checkpoints\1720262358040.ta.chkpt" at 2024-07-06 18:39:18 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 152 / 217 / 131 / 500: 100%|██████████████████| 500/500 [05:41<00:00,  1.47it/s]textattack: Saving checkpoint under "checkpoints\1720262363197.ta.chkpt" at 2024-07-06 18:39:23 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 152 / 217 / 131 / 500: 100%|██████████████████| 500/500 [05:41<00:00,  1.47it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 152    |
| Number of failed attacks:     | 217    |
| Number of skipped attacks:    | 131    |
| Original accuracy:            | 73.8%  |
| Accuracy under attack:        | 43.4%  |
| Attack success rate:          | 41.19% |
| Average perturbed word %:     | 14.72% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 21.01  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_rf_mr, n_samples, 'attack_results_new/clipmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   1%|▎                           | 5/500 [00:10<18:04,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1720262374264.ta.chkpt" at 2024-07-06 18:39:34 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 4 / 2 / 7:   1%|▍                           | 7/500 [00:11<13:01,  1.59s/it]

[Succeeded / Failed / Skipped / Total] 2 / 6 / 2 / 10:   2%|▌                         | 10/500 [00:14<12:04,  1.48s/it]textattack: Saving checkpoint under "checkpoints\1720262378093.ta.chkpt" at 2024-07-06 18:39:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 8 / 3 / 15:   3%|▊                         | 15/500 [00:19<10:15,  1.27s/it]textattack: Saving checkpoint under "checkpoints\1720262382343.ta.chkpt" at 2024-07-06 18:39:42 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 135 / 206 / 144 / 485:  97%|█████████████████▍| 485/500 [05:51<00:10,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720262714434.ta.chkpt" at 2024-07-06 18:45:14 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 137 / 208 / 145 / 490:  98%|█████████████████▋| 490/500 [05:55<00:07,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720262718387.ta.chkpt" at 2024-07-06 18:45:18 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 137 / 208 / 146 / 491:  98%|█████████████████▋| 491/500 [05:55<00:06,  1.38it/s]

[Succeeded / Failed / Skipped / Total] 137 / 210 / 148 / 495:  99%|█████████████████▊| 495/500 [05:58<00:03,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720262721419.ta.chkpt" at 2024-07-06 18:45:21 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 137 / 210 / 149 / 496:  99%|█████████████████▊| 496/500 [05:58<00:02,  1.38it/s]

[Succeeded / Failed / Skipped / Total] 138 / 213 / 149 / 500: 100%|██████████████████| 500/500 [06:01<00:00,  1.38it/s]textattack: Saving checkpoint under "checkpoints\1720262724825.ta.chkpt" at 2024-07-06 18:45:24 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 138 / 213 / 149 / 500: 100%|██████████████████| 500/500 [06:01<00:00,  1.38it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 138    |
| Number of failed attacks:     | 213    |
| Number of skipped attacks:    | 149    |
| Original accuracy:            | 70.2%  |
| Accuracy under attack:        | 42.6%  |
| Attack success rate:          | 39.32% |
| Average perturbed word %:     | 12.6%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 21.64  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_knn_mr, n_samples, 'attack_results_new/clipmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:10<16:37,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1720262735026.ta.chkpt" at 2024-07-06 18:45:35 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 5 / 2 / 10:   2%|▌                         | 10/500 [00:14<11:30,  1.41s/it]textattack: Saving checkpoint under "checkpoints\1720262739042.ta.chkpt" at 2024-07-06 18:45:39 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 8 / 3 / 15:   3%|▊                         | 15/500 [00:18<10:05,  1.25s/it]textattack: Saving checkpoint under "checkpoints\1720262743667.ta.chkpt" at 2024-07-06 18:45:43 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 167 / 166 / 152 / 485:  97%|█████████████████▍| 485/500 [05:34<00:10,  1.45it/s]textattack: Saving checkpoint under "checkpoints\1720263059137.ta.chkpt" at 2024-07-06 18:50:59 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 170 / 167 / 153 / 490:  98%|█████████████████▋| 490/500 [05:38<00:06,  1.45it/s]textattack: Saving checkpoint under "checkpoints\1720263063121.ta.chkpt" at 2024-07-06 18:51:03 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 170 / 167 / 154 / 491:  98%|█████████████████▋| 491/500 [05:38<00:06,  1.45it/s]

[Succeeded / Failed / Skipped / Total] 171 / 168 / 156 / 495:  99%|█████████████████▊| 495/500 [05:39<00:03,  1.46it/s]textattack: Saving checkpoint under "checkpoints\1720263064543.ta.chkpt" at 2024-07-06 18:51:04 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 171 / 168 / 158 / 497:  99%|█████████████████▉| 497/500 [05:39<00:02,  1.46it/s]

[Succeeded / Failed / Skipped / Total] 173 / 169 / 158 / 500: 100%|██████████████████| 500/500 [05:42<00:00,  1.46it/s]textattack: Saving checkpoint under "checkpoints\1720263067512.ta.chkpt" at 2024-07-06 18:51:07 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 173 / 169 / 158 / 500: 100%|██████████████████| 500/500 [05:42<00:00,  1.46it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 173    |
| Number of failed attacks:     | 169    |
| Number of skipped attacks:    | 158    |
| Original accuracy:            | 68.4%  |
| Accuracy under attack:        | 33.8%  |
| Attack success rate:          | 50.58% |
| Average perturbed word %:     | 9.05%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 20.99  |
+-------------------------------+--------+


In [31]:
runAttack(clip_model_wrapper_svc_mr, n_samples, 'attack_results_new/clipmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvcsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   1%|▎                           | 5/500 [00:09<16:20,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1720263077498.ta.chkpt" at 2024-07-06 18:51:17 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 4 / 2 / 7:   1%|▍                           | 7/500 [00:10<11:46,  1.43s/it]

[Succeeded / Failed / Skipped / Total] 4 / 4 / 2 / 10:   2%|▌                         | 10/500 [00:13<10:55,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720263080967.ta.chkpt" at 2024-07-06 18:51:20 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 6 / 3 / 15:   3%|▊                         | 15/500 [00:17<09:34,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720263085373.ta.chkpt" at 2024-07-06 18:51:25 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 155 / 205 / 125 / 485:  97%|█████████████████▍| 485/500 [05:43<00:10,  1.41it/s]textattack: Saving checkpoint under "checkpoints\1720263411387.ta.chkpt" at 2024-07-06 18:56:51 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 155 / 207 / 128 / 490:  98%|█████████████████▋| 490/500 [05:45<00:07,  1.42it/s]textattack: Saving checkpoint under "checkpoints\1720263413403.ta.chkpt" at 2024-07-06 18:56:53 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 156 / 210 / 129 / 495:  99%|█████████████████▊| 495/500 [05:49<00:03,  1.42it/s]textattack: Saving checkpoint under "checkpoints\1720263417122.ta.chkpt" at 2024-07-06 18:56:57 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 160 / 211 / 129 / 500: 100%|██████████████████| 500/500 [05:54<00:00,  1.41it/s]textattack: Saving checkpoint under "checkpoints\1720263421841.ta.chkpt" at 2024-07-06 18:57:01 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 160 / 211 / 129 / 500: 100%|██████████████████| 500/500 [05:54<00:00,  1.41it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 160    |
| Number of failed attacks:     | 211    |
| Number of skipped attacks:    | 129    |
| Original accuracy:            | 74.2%  |
| Accuracy under attack:        | 42.2%  |
| Attack success rate:          | 43.13% |
| Average perturbed word %:     | 14.94% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 21.01  |
+-------------------------------+--------+
